In [ ]:
import scipy.io as sio
import json
import sys
import numpy as np 
import pandas as pd 
import geopandas as gpd
import os 
import random
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.colors as pltcolors
import descartes 
import shapely.wkt 
import shapely.ops
import gc 
%matplotlib inline

absolute_path = '/home/bingyu/Documents/spatial_queue'

network_file_edges = '/projects/bolinas_stinson_beach/network_inputs/osm_edges.csv'
simulation_outputs = '/projects/bolinas_stinson_beach/simulation_outputs'
visualization_outputs = '/projects/bolinas_stinson_beach/visualization_outputs'

In [ ]:
def make_img(t=0, memory=None):

    ### Get edge veh counts
    
    road_df = pd.read_csv(absolute_path + network_file_edges)
    veh_df = pd.read_csv(absolute_path + simulation_outputs +'/veh_loc/veh_loc_{}s.csv'.format(t))
    
    road_gdf = gpd.GeoDataFrame(road_df, 
        crs={'init': 'epsg:4326'}, 
        geometry=road_df['geometry'].map(shapely.wkt.loads))
#     road_gdf = road_gdf.cx[-122.274061:-122.270398, 37.888713:37.890615]
    
    veh_gdf = gpd.GeoDataFrame(veh_df, 
                               crs={'init': 'epsg:4326'},
                               geometry=gpd.points_from_xy(veh_df.lon, veh_df.lat))
#     veh_gdf = veh_gdf.cx[-122.274061:-122.270398, 37.888713:37.890615]
    r = lambda: random.randint(0,255)
    
    def id_to_random_color(number, memory):
        if not number in memory:
            hexcolor = '#%02X%02X%02X' % (r(),r(),r())
            memory[number] = hexcolor
        return memory[number]
    
    veh_gdf['color'] = veh_gdf.apply(lambda x: id_to_random_color(x['v_id'], memory), axis=1)

    fig = plt.figure(figsize=(10, 10))
    
    ax1 = fig.add_axes([0.05, 0, 0.9, 0.9])
    ax1.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
    road_plot = road_gdf.plot(ax=ax1, lw=0.5, color=[0.7,0.7,0.7,0.5])
    veh_plot = veh_gdf.plot(ax=ax1, color=veh_gdf['color'])

    fig.text(0.5, 0.85, '{} sec into evacuation'.format(t), fontsize=30, ha='center', va='center')
#     plt.show()
    # sys.exit(0)
    plt.savefig(absolute_path + visualization_outputs + '/veh_loc_plot/veh_loc_t{}.png'.format(t), transparent=True)
    plt.close()
    
    return memory

In [ ]:
def make_gif(memory=None):

    import imageio

    images = []
    for t in range(100,150):
#         memory = make_img(t=t, memory=memory)
        images.append(imageio.imread('outputs/veh_loc_plot/veh_loc_t{}.png'.format(t)))
    imageio.mimsave('outputs/veh_loc_plot/veh_loc_animation.gif', images, fps=2)

In [ ]:
random.seed(0)
np.random.seed(0)
memory={}
make_img(t=80)
# make_gif(memory=memory)